In [1]:
import sys
from PyPDF2 import PdfReader
import os
import pandas as pd 
from dateutil import parser


In [3]:
folder_path = "facturas_pdf/1/"  # Specify the path to the folder

# Get the list of files in the folder
files = os.listdir(folder_path)

# Iterate over the files and print their names
for file_name in files:
    if "pdf" in file_name:
        print(file_name)

fv0890900608021230EA83277.pdf


In [4]:
# Get the file name from the command-line arguments
#Este para ejecutarlo si es un archivo py
#file_name = sys.argv[1]

#fv0890900608021230EA83277
#para ejecutarlo con el input
#file_name = input("Enter the PDF file name: ")
#Iterar a en el archivo y encontrar el nombre de el pdf
def nombre_pdf(folder_path, files):
    for file_name in files:
        if "pdf" in file_name:
            #print("-----------------------------------------------")
            #print(file_name)
            file_final = folder_path + file_name
    return(file_final)

#Funcion para extraer los datos 
def extraer_datos(file_final):
    # Open the PDF file using the provided file name
    with open(file_final, "rb") as file:
        pdf = PdfReader(file)
        for page in pdf.pages:
            #se extrae el texto del pdf
            text = page.extract_text()
            #print("texto :", text)
            #print("-----------------------------------------------")
    return(text)

#Funcion para eliminar los archivos iguales 
def eliminar_iguales(archivos, eliminar):
    for u in range(0, len(eliminar)):
        del archivos[eliminar[u]]
        #print('Se elimina este archivo :', eliminar[u])
    return(archivos)


def obtener_datos():
    archivos = []
    for n in range(1,98):
        #Specify the path to the folder
        folder_path = "facturas_pdf/" + str(n) + "/"  
        # Get the list of files in the folder
        files = os.listdir(folder_path)
        #Obtener el nombre del archivo
        file_final = nombre_pdf(folder_path, files)
        #Funcion para extraer los datos 
        text = extraer_datos(file_final)
        #Se separa por lineas y se inserta en la matriz 
        archivos.append(text.splitlines())
    return(archivos)

#Funcion para porcesar los datos y entregar series con los datos deseados 
def procesar_datos(archivos):
    fechas = []
    totales = []
    archivo = []
    mes = []
    eliminar = []
    for i in range(0,len(archivos)):
        for j in range(0,len(archivos[i])-1):
            #Como se separo por lineas se busca las palabras clave y el dato que le sigue seria el precio
            linea = str(archivos[i][j]).upper()
            linea_sig = str(archivos[i][j+1]).upper()
            #Se remueve el punto y se remplaza la coma por un punto para convertir a int
            linea_sig = linea_sig.replace('.','')
            linea_sig = linea_sig.replace(',','.')
            #Se hace un try para cambiar los valores de dinero a numeros 
            try:
                linea_sig = float(linea_sig)
            except: 
                pass

            #Condicones que identifican la fecha 
            if "FECHA:" in linea:
                fecha = linea_sig
                # Convert the date string to a datetime object
                fecha_obj = parser.parse(fecha)
                mes_obj = fecha_obj.month
                
            #Condiciones que identifican el precio final sin iva (porque es mas facil identificar este) y el iva 
            if "SUBTOTAL:" in linea:
                subtotal = linea_sig
                #print(linea, linea_sig)
            if "IVA:" in linea:
                iva = linea_sig
                #print(linea, linea_sig)

        #Se suma el subtotal y el iva para tener el total
        try:
            total = float(iva) + float(subtotal)
        except: 
            #En caso de no haber iva por ser un precio muy bajo como solo un pan
            total = float(subtotal)
        #print("Total:", total)
        #Se debe revisar si hay fechas repetidas
        if i > 0:
            if fecha in fechas:
                eliminar.append(i)
        fechas.append(fecha)
        totales.append(total)
        archivo.append(i + 1)
        mes.append(mes_obj)
    #Funcion para eliminar los archivos 
    archivos = eliminar_iguales(archivos, eliminar)
    return(archivos, fechas, totales, archivo, mes)

#Funcion para obtener el df
def crear_df_totales(fechas, totales, archivo, mes):
    datos = {'Fecha': fechas, 'Total': totales, 'Archivo': archivo, 'Mes': mes}
    df = pd.DataFrame(data=datos)
    #Eliminar duplicados 
    df = df.drop_duplicates(subset=['Fecha'])
    #Arreglar el index 
    df = df.reset_index()
    df = df.drop(['index'], axis=1)

    return(df)

#Funcion para obtener el df de los totales por mes
def crear_df_totales_mes(df):
    #Juntar todos los totales del mes 
    df2 = df.groupby(['Mes']).sum()
    #Se deben eliminar las columnas de archivos
    df2.drop(['Archivo'], axis=1, inplace=True)
    #df = df.drop(['index'], axis=1)
    return(df2)

#Funcion para obtener el df de solo las facturas mayores a 100.000
def crear_df_solo_mayores(df):
    datos = {'Total': [], 'Archivo': [], 'Mes': []}
    df3 = pd.DataFrame(data=datos)
    #Se deben insertar las facutras que sean de mas de 100.000
    for i in range(0,len(df['Total'])):
        try:
            if df.loc[i, 'Total'] >= 100000:
                df3.at[i, 'Total'] = df.loc[i, 'Total']
                df3.at[i, 'Archivo'] = df.loc[i, 'Archivo']
                df3.at[i, 'Mes'] = df.loc[i, 'Mes']
        except:
            pass
    return(df3)

#Funcion para identificar cual es el precio del prodcto 
def identificar_precio(archivos, i, j):
    #Se itera un loop que encontrara el precio 
    precios = []
    for k in range(0,15):
        precio = archivos[i][j+k]
        if '.' in precio and ',' in precio:
            precio = precio.replace('.','')
            precio = precio.replace(',','.')
            precio = float(precio)
            precios.append(precio)
        elif '.' not in precio and ':' not in precio and ',' in precio:
            precio = precio.replace(',','.')
            precio = float(precio)
            precios.append(precio)
    try:
        precio = max(precios)
    except ValueError: 
        precio = 0
    return(precio)

#Funcion para definir el total del precio del producto
def total_producto(precio, producto):
    producto = producto + precio
    return(producto)
    
#Funcion para crear dataframe con los totales por producto
def crear_df_totales_producto(archivos):
    #Todas las litas creadas aca deben seguir el orden correcto con las variables 
    clave_alim = [['LECHE'],['QUESO','CUAJADA','MANTEQUILLA'],['HUEVO'],['PAN ','BAGUET¾N'],['TABLA BJ',' RES ','DESMECHAR','PECHO BJ','BIFE'],
                  ['POLLO','PECHUGA','MUSLOS'],['CERDO','COSTILLA CERDO'],['AREPA','HARINA'],
                  ['BEBIDA','JUGO','GASEOSA', 'YOGURT', 'HATSU', 'TETRAPACK'],['JAMON','TOCINO','SALCHI','JAM¾N'],['ARROZ','FRIJOL','GARBANZO'],
                  ['ARRACACHA','ALBAHACA','HABICHUELA','AHUYAMA','JENGIBRE','MAZOROCA','CHOCOLO','RABANO','TOMATE','PEPINO','APIO',
                   'LECHUGA','CILANTRO','CHAMPI','PIMENTON','BROCOLI','PAPA','ESPINACA','ZANAHORIA','KALE','CEBOLLA','REPOLLO','COLIFLOR'],
                   ['MANZANA','MANGO','HIGOS','PLATANO','BANANO','AGUACATE','KIWI','PERAS','GUAYABA','MORA','PAPAYA','GRANADILLA','NARANJA',
                    'COCO','LULO','LIMON','LIM¾N','MANDARINA','UVA','DURAZNO'],['PESCADO','SALMON','TILAPIA','ATUN']]
    clave_prod = [['PAPEL','HIGIENICO','HIGIENICA'],['JABON','DENTAL','SHAMPOO'],['COCINA','SERVILLETAS','LAVALOZA'],
                  ['PANTALON','COBIJA','ALMOHADA','CAMISA','CAMISETA','BUSO','PANTAL¾N'], ['GIRASOL'], ['ACEITE PREMIUM', 'OLIVA'],
                  ['CREMA','ANTI'],['SAL ','AZUCAR'],['VINO','V.BLANCO'],['BICARBONATO','MAIZITOS','ROSQUITAS','GALLETAS']]

    leche, queso, huevos, pan, carne, pollo, cerdo, harina, bebidas, jamon, granos, vegetales, fruta, pescado = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    papel, aseo_personal, aseo_cocina, ropa, aceite_veg, aceite_oliva, cremas, sal_azucar, vino, otros = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    valores_alim = [leche, queso, huevos, pan, carne, pollo, cerdo, harina, bebidas, jamon, granos, vegetales, fruta, pescado]
    valores_prod = [papel, aseo_personal, aseo_cocina, ropa, aceite_veg, aceite_oliva, cremas, sal_azucar, vino, otros]
    
    nombres_alim = ['Leche', 'Queso/Cuajada', 'Huevos', 'Pan', 'Carne Res', 'Pollo', 'Carne Cerdo', 'Harinas/Arepas', 'Bebidas', 
                    'Jamon/Tocineta', 'Granos', 'Vegetales', 'Frutas', 'Pescado']
    nombres_prod = ['Papel Higienico', 'Aseo Personal', 'Aseo Cocina', 'Ropa', 'Aceite Vegeta', 'Aceite Oliva', 'Cremas', 
                    'Sal/Azucar', 'Vino', 'Otros']

    
    df5_A = pd.DataFrame(data={'Leche': [], 'Queso/Cuajada': [], 'Huevos': [], 'Pan': [], 'Carne Res': [], 'Pollo': [], 
                                 'Carne Cerdo': [], 'Harinas/Arepas': [], 'Bebidas': [], 'Jamon/Tocineta': [], 'Granos': [], 
                                 'Vegetales': [], 'Frutas': [], 'Pescado': []})
    
    df5_B = pd.DataFrame(data={'Papel Higienico': [], 'Aseo Personal': [], 'Aseo Cocina': [], 'Ropa': [], 'Aceite Vegeta': [], 
                               'Aceite Oliva': [], 'Cremas': [], 'Sal/Azucar': [], 'Vino': [], 'Otros': [], 'Mes': []})
    

    for i in range(0, len(archivos)):
        #print('Archivo :', archivos[i])
        #Para hacer el mensual 
        valores_alim_ar = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        valores_prod_ar = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        for j in range(0,len(archivos[i])):
            #print(archivos[i][j])
            linea = str(archivos[i][j]).upper()
            
            #Condicones que identifican la fecha 
            if "FECHA:" in linea:
                #print("-----------------------------------------------")
                linea_sig = str(archivos[i][j+1]).upper()
                fecha = linea_sig
                # Convert the date string to a datetime object
                fecha_obj = parser.parse(fecha)
                mes_obj = fecha_obj.month
                #Agregar mes a la fila
                df5_B.at[i, 'Mes'] = mes_obj

            ### Loop para los alimentos ----------------------------------------------------------
            for u in range(0,len(clave_alim)):
                for r in range(0,len(clave_alim[u])):
                    if clave_alim[u][r] in linea:
                        precio = identificar_precio(archivos, i , j)
                        producto = total_producto(precio, valores_alim[u])
                        producto_ar = total_producto(precio, valores_alim_ar[u])
                        valores_alim[u] = producto
                        valores_alim_ar[u] = producto_ar
                        #agregar precios a df5
                        df5_A.at[i, nombres_alim[u]] = valores_alim_ar[u]
                        #print(archivos[i][j], ', Precio :', precio)

            ### Loop para los Productos ----------------------------------------------------------
            for t in range(0,len(clave_prod)):
                for q in range(0,len(clave_prod[t])):
                    if clave_prod[t][q] in linea:
                        precio = identificar_precio(archivos, i , j)
                        producto = total_producto(precio, valores_prod[t])
                        producto_ar = total_producto(precio, valores_prod_ar[t])
                        valores_prod[t] = producto
                        valores_prod_ar[t] = producto_ar
                        #agregar precios a df5
                        df5_B.at[i, nombres_prod[t]] = valores_prod_ar[t]
                        #print(archivos[i][j], ', Precio :', precio)
    #Modificar lo dataframes    
    df_A = pd.DataFrame(data={'Alimentos': [], 'Precio alimentos': []})
    df_P = pd.DataFrame(data={'Productos': [], 'Precio productos': []})
    for n in range(0,len(valores_alim)):
        df_A.at[n, 'Alimentos'] = nombres_alim[n]
        df_A.at[n, 'Precio alimentos'] = valores_alim[n]
    for y in range(0,len(valores_prod)):
        df_P.at[y, 'Productos'] = nombres_prod[y]
        df_P.at[y, 'Precio productos'] = valores_prod[y]
    df4 = pd.merge(df_A, df_P, left_index=True, right_index=True, how='left')
    df5 = pd.merge(df5_A, df5_B, left_index=True, right_index=True, how='left')
    #Agrupar los productos por mes 
    df6 = df5.groupby(['Mes']).sum()
    return(df4, df5, df6)

#Funcion para crear el excel 
def crear_excel(df, df2, df3, df4, df5, df6, nombre):
    # Specify the output Excel file path
    output_file = nombre
    with pd.ExcelWriter(output_file) as writer: 
        df.to_excel(writer, sheet_name='Todas')
        df2.to_excel(writer, sheet_name='Totales-Mes')
        df3.to_excel(writer, sheet_name='Solo Grandes')
        df4.to_excel(writer, sheet_name='Totales Producto')
        df5.to_excel(writer, sheet_name='Totales Producto por Archivo')
        df6.to_excel(writer, sheet_name='Totales Producto por Mes')

#Funciones del proceso 

#extraer los datos de los arhcivos pdf en una lista de python donde cada lista dentro de esta es un archivo
archivos = obtener_datos()
#Extraer de estos datos la fecha, los totales de precio, el numero del archivo donde se encuntra la facutra
archivos, fechas, totales, archivo, mes = procesar_datos(archivos)
#Crear un data frame a aprtir de estos datos
#Precios y totales por factura 
df = crear_df_totales(fechas, totales, archivo, mes)
#totales por mes 
df2 = crear_df_totales_mes(df)
#totales de facturas meyores
df3 = crear_df_solo_mayores(df)
#crear DF's de totales por producto, producto por mes, y producto por factura
df4, df5, df6 = crear_df_totales_producto(archivos)
#Crear excel
crear_excel(df, df2, df3, df4, df5, df6, 'Datos_sin_sumaTotales.xlsx')
#Funcion para hacer prueba de resultados 


C:\Users\sebas\AppData\Local\Temp\ipykernel_27048\2870368354.py:251: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Leche' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_A.at[n, 'Alimentos'] = nombres_alim[n]
C:\Users\sebas\AppData\Local\Temp\ipykernel_27048\2870368354.py:254: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Papel Higienico' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_P.at[y, 'Productos'] = nombres_prod[y]
